In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 有效广告及其特征

In [2]:
import pandas as pd
from pathlib import Path
import os, re
from functools import reduce, partial
from utils import or_func, add_datepart, split_vals

In [3]:
root = Path('../data')
root
# os.listdir(root)

PosixPath('../data')

In [4]:
users = pd.read_csv(root/'users_data.csv', encoding='utf-8')
users.head()
users.info()

,用户id,Age,Gender,地域,婚恋状态,学历,消费能力,设备,工作状态,连接类型,行为兴趣
0,624218,753,2,"523,7589,12527,9019,14605,13489,302,6222,14214...",0,6,1,2,0,4,"32612,34406,3413,33059,4275,4667,4561,34051,42..."
1,271475,601,3,"9019,9032,663,12527,14532,2416,2599,1987,9687,...",13,7,2,2,0,2,"4990,4992,3413,33061,4115,4876,2393,34167,3441..."
2,44439,819,3,"12527,14188,9841,9019,14605,6222,9962,6643,172...",13,7,3,2,4,4,"3099,4403,34414,431,34415,5108,31686,3240,3229..."
3,863717,1,2,"12527,3435,2398,11069,1292,9019,14605,14348,62...",6,5,3,2,0,2,"34293,845,3413,3099,33061,33059,4120,4410,4415..."
4,1305469,714,3,"8928,9019,1540,719,12527,6051,10545,12118,7107...",13,5,3,2,0,2,"32612,34289,34406,3413,4275,4410,4561,34051,34..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396718 entries, 0 to 1396717
Data columns (total 11 columns):
用户id      1396718 non-null int64
Age       1396718 non-null int64
Gender    1396718 non-null int64
地域        1396718 non-null object
婚恋状态      1396718 non-null object
学历        1396718 non-null int64
消费能力      1396718 non-null int64
设备        1396718 non-null int64
工作状态      1396718 non-null object
连接类型      1396718 non-null int64
行为兴趣      1396718 non-null object
dtypes: int64(7), object(4)
memory usage: 117.2+ MB


In [5]:
test = pd.read_csv(root/'test_data.csv', encoding='utf-8')
test.head()
test.info()

,样本id,广告id,创建时间,素材尺寸,广告行业id,商品类型,商品id,广告账户id,投放时段,人群定向,出价
0,1,394352,1529648412,34,84,13,29663,26657,"281474976645120,281474976645120,28147497664512...","age:819,608,988,741,202,837,400,394,942,361,72...",120
1,2,585401,1553076190,40,221,1,-1,6262,"281474976579587,281474976579587,28147497657958...","age:819,433,479,741,229,347,522,79,753,601|edu...",42
2,3,419408,1553031394,30,122,13,32110,17436,"17592185782272,17592185782272,17592185782272,1...",all,6
3,4,405326,1553238836,64,136,1,-1,22359,"281474976694272,281474976694272,28147497669427...","age:333,1|gender:2|area:11505,1874,3790,4566,5...",181
4,5,578942,1541191585,34,12,13,6372,24082,"68719214592,68719214592,68719214592,6871921459...","age:819,608,988,741,202,837,400,394,942,361,72...",31


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20290 entries, 0 to 20289
Data columns (total 11 columns):
样本id      20290 non-null int64
广告id      20290 non-null int64
创建时间      20290 non-null int64
素材尺寸      20290 non-null int64
广告行业id    20290 non-null int64
商品类型      20290 non-null int64
商品id      20290 non-null int64
广告账户id    20290 non-null int64
投放时段      20290 non-null object
人群定向      20290 non-null object
出价        20290 non-null int64
dtypes: int64(9), object(2)
memory usage: 1.7+ MB


### 多值特征的处理
- 离散为 bool 特征

In [6]:
users.shape
test.shape

(1396718, 11)

(20290, 11)

## 测试数据

In [7]:
cols = ['age','area','behavior','connectionType','consuptionAbility','device','education','gender','status','work']
test['is_all_field'] = test['人群定向'] == 'all'
test['time'] = test['投放时段']
test.drop(columns='投放时段', inplace=True)
test['创建时间'] = pd.to_datetime(test['创建时间'], unit='s')
test['has_product_id'] = test['商品id'] != -1  # - 1 表示的无商品 id
test = test.reindex(columns=test.columns.to_list() + cols)

### 人群定向的分解

In [8]:
func = partial(split_vals, cols=cols, field='人群定向')

In [9]:
test = test.apply(func, axis=1)
test.drop(columns='人群定向', inplace=True)

- -999 表示空值

In [10]:
test.fillna(-999, inplace=True)

In [11]:
test.head().T

,0,1,2,3,4
样本id,1,2,3,4,5
广告id,394352,585401,419408,405326,578942
创建时间,2018-06-22 06:20:12,2019-03-20 10:03:10,2019-03-19 21:36:34,2019-03-22 07:13:56,2018-11-02 20:46:25
素材尺寸,34,40,30,64,34
广告行业id,84,221,122,136,12
商品类型,13,1,13,1,13
商品id,29663,-1,32110,-1,6372
广告账户id,26657,6262,17436,22359,24082
出价,120,42,6,181,31
is_all_field,False,False,True,False,False


- 时间特征

In [12]:
add_datepart(test, '创建时间', drop=True, time=True)

In [13]:
test.head().T

,0,1,2,3,4
样本id,1,2,3,4,5
广告id,394352,585401,419408,405326,578942
素材尺寸,34,40,30,64,34
广告行业id,84,221,122,136,12
商品类型,13,1,13,1,13
商品id,29663,-1,32110,-1,6372
广告账户id,26657,6262,17436,22359,24082
出价,120,42,6,181,31
is_all_field,False,False,True,False,False
time,"281474976645120,281474976645120,28147497664512...","281474976579587,281474976579587,28147497657958...","17592185782272,17592185782272,17592185782272,1...","281474976694272,281474976694272,28147497669427...","68719214592,68719214592,68719214592,6871921459..."


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20290 entries, 0 to 20289
Data columns (total 37 columns):
样本id                        20290 non-null int64
广告id                        20290 non-null int64
素材尺寸                        20290 non-null int64
广告行业id                      20290 non-null int64
商品类型                        20290 non-null int64
商品id                        20290 non-null int64
广告账户id                      20290 non-null int64
出价                          20290 non-null int64
is_all_field                20290 non-null bool
time                        20290 non-null object
has_product_id              20290 non-null bool
age                         20290 non-null object
area                        20290 non-null object
behavior                    20290 non-null object
connectionType              20290 non-null object
consuptionAbility           20290 non-null object
device                      20290 non-null object
education                   20290 non-null object
gen

In [15]:
test.shape

(20290, 37)

In [16]:
test.columns

Index(['样本id', '广告id', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id', '出价',
       'is_all_field', 'time', 'has_product_id', 'age', 'area', 'behavior',
       'connectionType', 'consuptionAbility', 'device', 'education', 'gender',
       'status', 'work', 'crt_dateYear', 'crt_dateMonth', 'crt_dateWeek',
       'crt_dateDay', 'crt_dateDayofweek', 'crt_dateDayofyear',
       'crt_dateIs_month_end', 'crt_dateIs_month_start',
       'crt_dateIs_quarter_end', 'crt_dateIs_quarter_start',
       'crt_dateIs_year_end', 'crt_dateIs_year_start', 'crt_dateHour',
       'crt_dateMinute', 'crt_dateSecond', 'crt_dateElapsed'],
      dtype='object')

In [17]:
test['price'] = test['出价']
test.drop(columns=['出价'], inplace=True)

In [18]:
test.to_csv('../data/test_set.csv', index=None, encoding='utf-8')

- fin 测试集初步处理